In [10]:
### Mount Google Drive ###

from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
!scp '/content/gdrive/MyDrive/Projects/AI/CV/text-detection-python-tesseract-easyocr-textract/data.zip' '/content/data.zip'

!unzip '/content/data.zip' -d '/content/'

Archive:  /content/data.zip
   creating: /content/data/
  inflating: /content/data/I_hate_nothing_about_u.jpg  
  inflating: /content/data/I_see_a_light_in_the_darkness.jpg  
  inflating: /content/data/aloha.jpg  
  inflating: /content/data/art.jpg   
  inflating: /content/data/aspire_to_inspire.jpg  
  inflating: /content/data/be_kind.jpg  
  inflating: /content/data/be_kind_to_one_another.jpg  
  inflating: /content/data/be_my_valentine.jpg  
  inflating: /content/data/beer_beer_beer.jpg  
  inflating: /content/data/berlin.jpg  
  inflating: /content/data/black_lives_matter_no_racism_love_not_war.jpg  
  inflating: /content/data/blank_street.jpg  
  inflating: /content/data/brand_personality_&_visual_identity.jpg  
  inflating: /content/data/burn_fat.jpg  
  inflating: /content/data/can't_hide_our_pride.jpg  
  inflating: /content/data/caution_maintain_social_distancing_at_least_6ft_distance_from_other.jpg  
  inflating: /content/data/collect_moments_not_things.jpg  
  inflating: /co

In [12]:
!apt install tesseract-ocr
!apt install libtesseract-dev

!pip install pytesseract
!pip install Pillow
!pip install easyocr
!pip install boto3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,489 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123624 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

# Text Detection

In [31]:
import pytesseract
from PIL import Image
from easyocr import Reader
import boto3


reader = Reader(['en'])

acces_key = 'AWS_ACCESS_KEY_ID'
secret_acces_key = 'AWS_SECRET_ACCESS_KEY'

textract_client = boto3.client('textract',
                             aws_access_key_id=acces_key,
                             aws_secret_access_key=secret_acces_key,
                             region_name='us-east-1')


def read_text_tesseract(image_path):
  text = pytesseract.image_to_string(Image.open(image_path), lang='eng')
  return text

def read_text_easyocr(image_path):
  text = ''
  results = reader.readtext(Image.open(image_path))

  for result in results:
    text = text + result[1] + ' '

  text = text[:-1]
  return text

def read_text_textract(image_path):
  with open(image_path, 'rb') as im:
    response = textract_client.detect_document_text(Document={'Bytes' :im.read()})

    text = ''
    for item in response['Blocks']:
      if item['BlockType'] == 'LINE':
        text = text + item['Text'] + ' '

  text = text[:-1]
  return text

# Compare Performances

In [34]:
import os


def jaccard_similarity(sentence1, sentence2):
    # Tokenize the sentences by splitting on whitespace
    set1 = set(sentence1.lower().split())
    set2 = set(sentence2.lower().split())

    # Calculate the intersection and union
    intersection = set1.intersection(set2)
    union = set1.union(set2)

    # Calculate Jaccard Similarity
    if len(union) == 0:
        return 0  # To handle the case where both sets are empty
    return len(intersection) / len(union)



score_tesseract = 0
score_easyocr = 0
score_textract = 0

for image_path_ in os.listdir('/content/data'):
  image_path = '/content/data/' + image_path_

  gt = image_path[:-4].replace('_', ' ').lower()

  score_tesseract += jaccard_similarity(gt, read_text_tesseract(image_path).lower().replace('\n', '').replace('!', '').replace('?', '').replace('.', ''))
  score_easyocr += jaccard_similarity(gt, read_text_easyocr(image_path).lower().replace('\n', '').replace('!', '').replace('?', '').replace('.', ''))
  score_textract += jaccard_similarity(gt, read_text_textract(image_path).lower().replace('\n', '').replace('!', '').replace('?', '').replace('.', ''))



print('score tesseract: ', score_tesseract / len(os.listdir('/content/data')))
print('score easyocr: ', score_easyocr / len(os.listdir('/content/data')))
print('score textract: ', score_textract / len(os.listdir('/content/data')))


score tesseract:  0.011739785068732436
score easyocr:  0.22276386375921972
score textract:  0.3444905058391903
